In [16]:
import pandas as pd

import gtsam

In [14]:
traj = pd.read_csv("../results/kitti_superpoint_supergluematch.txt",header=None, delimiter=" ",names=["id","x","y","z","roll","pitch","yaw"])

In [15]:
traj

,id,x,y,z,roll,pitch,yaw
0,0,0.000000,0.000000,0.000000,5.551115e-17,3.330669e-16,-4.440892e-16
1,1,-0.000441,0.004049,0.860433,-4.690294e-02,-2.839928e-02,8.586941e-01
2,2,-0.039375,-0.040310,1.717731,-9.374345e-02,-5.676064e-02,1.716275e+00
3,3,-0.061143,-0.036502,2.577885,-1.406429e-01,-8.515762e-02,2.574964e+00
4,4,-0.100830,-0.044864,3.436358,-1.874858e-01,-1.135202e-01,3.432648e+00
...,...,...,...,...,...,...,...
4536,4536,-44.755349,334.836454,97.858129,-5.369648e+00,-3.459248e+00,9.243554e+01
4537,4537,-44.613800,334.936302,98.978268,-5.418986e+00,-3.482548e+00,9.356768e+01
4538,4538,-44.523155,335.048932,100.103523,-5.470861e+00,-3.498163e+00,9.470089e+01
4539,4539,-44.378256,335.136901,101.219051,-5.524006e+00,-3.526732e+00,9.582761e+01
